# Relevance in Deeplinked Embedded Ads for URX

The goal of this project is to construct a model that can effectively predict the relevance between a publisher's webpage and an advertiser's webpage. 

## Setup:

### Background: 

There are two general strategies that advertisers can employ to capture potential customers' attention. The first strategy is to find the apppropriate context in which the customer will be most amemnable to receiving the advertisement. This is like putting a billboard advertising bottled water in the middle of the desert. The second strategy is to make an advertisement which is sufficiently entertaining that your customer won't mind (or perhaps even realize) that the ad isn't relevant to the context. This is putting a billboard with a hilarious jokes and lots of nudity promoting a new movie in the desert. 

In the twentieth century, as we developed broadcast-based mass media, the same opportunities that allowed brands to advertise to massive audiences did not allow them to target the audience when they're most ammenable to their products' value propositions. Most people are not actively thinking about a new car while watching their favorite television program, and this fact of circumstance has forced the advertising industry to focus on creative over relevance. 

With the Internet and increasingly sophisticated targeting, demographic, and context information, relevance is now an increasingly effective strategy for advertising. 

URX is an advertising technology startup which has focused almost exclusively on advertising by context: all of their ads pretty much look the same. 

### Two challenges: defining relevance and a lack of ground truth

Relevance is subjective and depends on the beliefs and information state of the customer. Even in what might seem like the most obvious situations, relevance is subjective. For the man crawling through the desert, bottled water might seem like an obviously relevant ad, but it might be less relevant than a subscreen ad. Second, relevance is not directly observable. You can observe views, loads, clicks, actions, and a variety of other  metrics in online marketing, but not whether an ad was relevant. Of course, more relevant ads should be clicked more frequently, but there is no way to determine whether the higher click rate is a matter of relevance, ad copy, picture, or just novelty. 

URX has decided to handle the challenges of relevance in two ways. The first is that they've defined relevance with great precision in order to improve intersubjective agreement. Two pages are relevant if the publisher's page is about the same named entity as the advertiser's page. A named entity is essentially a proper noun: Michael Jackson, Goldman Sachs, the Stanley Cup. They've decided to overcome the fact that relevance is not directly observable by using a crowdsourced human intelligence platform, Crowd Flower, which makes a living by tagging data sets to convert unsupervised learning tasks into supervised ones. 

### Experimental Design

Ben Bowles at URX designed and ran two experiemnts with Crowd Flower. Because URX has two different algorithms to select a relevant ad, for each publisher URL, there are two different advertiser URL's with each publisher URL (excpet some for which the second URL was lost from the data). In total, the experiment contains 950 distinct publisher/advertiser pairs. For each pair, approximately 10 workers were asked to rank the relevance of the publisher/advertiser webpages on a scale of 1 to 3. 1 indicates that the two are irrelvant, that is, they are not about the same named entities. 3 indicates that they are both about the same named entities. 2 indicates that the pages do not contain the same named entities, but there is a named entitity connection that can be made, for example, one page is about Kurt Cobain and the other about Nirvana, and since Cobain is the singer of Nirvana, there is a named entity connection, but it is not obvious. Some workers might have used 2 to indicate when they were confused.


## Insights

This project has a couple of insightful themes. 
1. Relevance, as defined by URX, is machine learnable with a high degree of accuracy. 
2. The performance of my model is high across the entire data set, and for any given publisher web domain. 
3. I employed a simple method to detect fraud from worker features that doesn't require any parameter tuning. 

# Overview of the project  

There are three stages in my pipeline. I include  dead ends in my process when interesting.  

1. Preprocessing the data 
        1. Detecting fraud
        2. Aggregating the relevance scores of different workers
        3. Extracting and engineering features from the publisher and advertiser webpage 

2. Training the model
        A. Choosing the model 
 
3. Validating the model 
        A. Performance metrics 

## 1. Preprocessing the data 

In order to avoid garbage in, garbage out, the input data needs to be preprocessed and checked for quality. There were two problems with the quality of the scores that the workers provided. Some workers were fraudulent robots, or if not fradulent robots, as good as fradulent robots. It was critical to remove their judgments to ensure a high quailty prediction.

The publisher and ads had only their urls, so I needed to scrape the webpages and extract meaningful features. First I loaded the data set from CSV and removed the 'golden' rows (those used for quality assurance) and the rows with blank entries. 

Link to [0 preprocess csv](0 preprocess csv v2.ipynb)


### 1.1 Detecting Fraud:

Fraud detection is a well-established problem in data science. Supervised learning approaches require a labeled data set in order to train a prediction algorithm. While my data clearly contained fraud, I did not have ground truth labeled data to train a model. This leaves me with an unsupervised learning task. There are essentially two appraoches in a situation like this: clustering and anomaly detection. In clustering, you attempt to suss out groups (fraud and not-fraud) inside some feature space of the observations. In anomaly detection, you establish some 'normal' range that you observations ought to fall into, and then identify those which fall above or below the normal thresholds. I used both. Before applying either of these approaches, however, you need to have some kind of feature space. 

##### Feature space

    1. Job count per worker

As your intution would indicate, the distribution of jobs per worker looks approximately log normal. That is, there is a concentrated lump, and a long right tail. However, some workers take on as many jobs as they can. Perhaps they enjoy the task, or as they improve their productivity, it becomes more profitable than novel tasks. Again, consistent with intution, fraudulent workers take on more jobs. So the observed distribution of jobs per worker looks like a log normal distribution with a big spike toward the end where a small number of workers (but much more than you would expect) took on an enormous number of jobs. Some of those in the tail spike are frauds, however high job count does not guarantee fraud. 

    2. Score variance per worker

Assuming that workers were given jobs randomly, the distribution of scores per worker should be, with a margin of error, approximately the same as that of publisher/advertiser relevance scores as a whole. So you would expect the variance of different workers to be approximately normally distributed. Some workers, however, had a variance of zero. That is, they entered the same score for every single job they did.


    3. Mean score per worker

Similar to the score variance per worker, given the global mean and the number of jobs a worker takes on, we can establish a 95% confidence ratio for the mean score per worker. Since we can't establish the global mean with aggregating the worker's scores, and we can't aggregate the worker's scores without knowing which workers to exclude becuase of fraud, this measure was treated like all of the other measures. 

    4. Time stamp difference regularity

The Crowd Flower's data contains timestamp information about the datetime for the creation of each job, and for when the worker actually started the job. The distribution of the difference between these timestamps should be normally distributed around the average amount of time it takes the worker to get started. However, when you look at the actual histograms, some workers have a nice normal distribution, but others have a strinking regularity to how long it takes them to start the job, indicative of an automated script (bot). 



However, this regularity isn't going to be captured by a simple measure on the distribution of timestamp differences such as the mean or the variance. In order to capture this, I took the number of unique timestamp difference values over the total number of jobs that the worker did. We can see that this measure, looking across all of the workers, is approximately normal, with the suspiciously regular timestamp differences falling outside of the 95% confidence interval. 

    5. Frequency of minority judgments 

Worker's aren't always going to be correct. Let's call a judgment correct if it falls into the majority and wrong if it falls into the minority. 

###### Cultural confusion? 
I also ggregated minority judgments at the country level as opposed to the worker level. The thinking here is that a worker might be in the minority on too many votes by one of three mechanisms: either the worker is a mindles robot, the worker is a thoughtless fraud, or the worker does not understand the nature of the task being done. In all three cases this produces low quality data. Since the instructions and website are all in English, I want the feature to account for the last possibility.  As should be pretty obvious, we can't account for the minority judgments as a result of confusion when one of the highest rates of minority judgmnets is the United States. Moreover, suppose that you elimate fraud and error using the first four judgments (and the sumple sum and threshold aggregation approach), and then check how many further jobs are removed by removing the fraudulent countries, this is only 1-2% of the jobs. That is, fraudulent countries are not fradulent becuase of a few bad players. 

####  Binarizing the features

I could have fit each of the features with a distribution and then calculated the parameters, but this is problematic for two reasons. First, it's unclear what distribution some of the featuers ought to follow (normal? log normal? truncated normal? zero inflated?), and then much information is lost in reducing all of the information in the features' distributions into parameters. Moreover, it requires a non-trivial amount of manual effort to stare at, fit, and test different distributions for each parameter. I opted to develop a very basic, but robust, non-parametric bootstrapped anomaly detection approach. That is, I resampled each of the distributions from the observations, which should over lots of sampling be normally distributed, and then calculatd a 95% confidence interval. A given worker would be given a 0 for a feature if she fell within this interval, and 1 otherwise. 

##### Aggregating the fraud features

In order to aggregate these five dimensions of fraud, I used a simple k-means clutering algorithm with two nodes, and let it do its work. I visually compared this unsupervised result with a cruder approach: adding them up and setting a threshold. The neat thing about the unsupervised result is that it requires a minimal amount of manual parameter twiddling. A person needs to come up with the features, but from there one, because of the bootstrapping and the clustering, the only thing that neeeds to be done is set the number of clusters to 2. 

I'm now going to show sets of plots of each feature, colored by the two appraoches (thresholding and clustering algorithm):

Histogram of job counts per worker, colored by thresholding
<img src="http://noahburbank.com/files/crude%20count.png">
Histogram of job counts per worker, colored by clutering
<img src="http://noahburbank.com/files/algo%20count.png">

Histogram of variance per worker, colored by thresholding
<img src="http://noahburbank.com/files/variance%20crude.png">
Histogram of variance per worker, colored by clustering
<img src="http://noahburbank.com/files/variance%20algo.png">

Histogram of mean per worker, colored by thresholding
<img src="http://noahburbank.com/files/mean%20crude.png">
Histogram of mean per worker, colored by clustering
<img src="http://noahburbank.com/files/mean%20algo.png">

Histogram of diversity of time stamp difference per worker, colored by thresholding
<img src="http://noahburbank.com/files/diversity%20crude.png">
Histogram of diversity of time stamp difference per worker, colored by clustering
<img src="http://noahburbank.com/files/diversity%20algo.png">

Histogram of minority judgments, colored by thresholding
<img src="http://noahburbank.com/files/minority%20crude.png">
Histogram of minority judgments per worker, colored by clustering
<img src="http://noahburbank.com/files/minority%20algo.png">

You can see that the clustering approahc does a much better job of identifying outliers than the simple thresholding appraoch, especially with everything except job count. 

For more please see:  [1 worker evaluation](2 worker evaluation v2.ipynb)

### 1.2 Aggregating the relevance scores of workers: 

There are two general approaches to aggregating the relevance score, each of which reflects a different underlying concept of relevance. You can take the average of the votes, and end up with some value between 0 and 1, inclusive, or you can have a winner-take-all vote system in which the relevance value for each publisher/ad pair is either 0 or 1. The first method looks at relevance as the term is used in normal, everyday English. The problems with this are twofold: this definition of relevance is highly subjective, and it contradicts the precise definition (being about the same named entity) that URX is using. This basically leaves us with the second approach, however there are two general approaches to aggregating the individual worker judgments into a binary. The first is to go with a majority vote (perhaps throwing out the publisher/advertiser pairs for which there is not a high enough consensus), and the second is to take the mean and then round up or down contingent on some threshold. I opted for the first approach becuase Ben told me that when the majority consisted of at least ~60%, they tended to get the right answer. This also is a point in favor of the minority worker fraud detection feature. As you can tell, once the fraud is removed, there is typically a decent concensus around relevance judgments. 

<img src="files/histogram of plurality percentage exp2.png">

In order to keep things consistent, I would throw out the pub/ad pairs for which there was not a 60% majority. After cleaning out the problematic workers, this led to a less than 5% reduction in pub/ad pairs. I also checked for pub/ad pairs for which 2 was the plurality vote, but they were all thrown out with the same 60% threshold, so this ended up being a moot point. 

##### A couple of deadends

I tried aggregating by taking the average, which was both inconsistent with the definition of relevance in the task definition, and ruined the classification nature of the task. It also was atrocious for performance: most people agreed about when something was irreelvant, but then the distribution was basically flat between zero and one. I tried getting back to a classification task by rounding up/down to 1/0, but the cutoff threshold was arbitrary and the process was no longer clearly intepretable. 

For more, please see [3 score aggregation](3 score aggregation v2.1.ipynb)

### 1.3 Feature Engineering

The original data set contains very little information about the publisher and advertiser webpages. In fact, it contained only their urls. In order to engineer features, I first scraped each webpage. I scraped the titles, the page description/metatags, and the bodies of the websites. For more on the details of this, please see my code on page [insert a link to the page here]. In order to remove the javascript and other automatically generated nonsense that accompanies most modern webpages, I employed a two-step filter. The first step remove elements of pages with nonsense characters ({, }, / }, etc.) above a threshold, and the second removed elements of pages without sufficient character count. Reading the output, this seemed to work out pretty well. The features that I engineered were the following: 

1. Cosine similarity on the tf-idf of the title, description, and body

Tf-idf, or term frequency-inverse document frequency, is a matrix with documents as rows and terms as columns. When each term appears in a document, it is given a value which increased based on the number of times that it appears in the document, and which is reduced by the frequency with which the word appears in other documents. This score ranges between 0 and 1, and increases for words which are, generally speaking, more important for the document. The cosine similarity compares two document by multiply their values for each document and adding them all up and ranges between 0 and 1. Documents that are more similar have higher scores, and identical documents have a score of 0. I removed extremely common words using the python Natural Language ToolKit (nltk) package's stopwords for english, and stemmed (removed suffices, pluralities, etc. to find common word roots) again using nltk. 

2. One-, two-, and three-gram similarity

Ngrams are N length sequences of words. I used the ngram packages to calculate similarity (a score of between 0 and 1 with 1 be identical) for seuqences of one, two, or three words in length for the title, description, and the body of the webpages. 

3. Longest common string

The longest common string algorithm calculates the longest substring that two strings have in common. This often returned a nonsense string, so in order to make sure that this longest common substring was meaningful, I tokenized (divided the string into individual words), removed stopwords (common words like 'the' and 'and'), and then ensured that the word in the substring had at least three characters. I then removed the words that appeared too frequently (ex. 'all' 'rights' 'reserved') to be meaningful, and then created a simple binary variable based on an intersection of longest common string between the publisher and ad pages. I also employed tf-idf on the longest common string, since it woudl capture, algorithmically, much of what I did with the intersection method, although more formally. The proved about equally effective. 

4. Named Entity Recognition (NER)

I used to the Stanford-NER algorithm to extract named entities from the title, description, and body of the webpages. I then converted this back into a string, constructed the tf-idf matrix, and calculated cosine similarity based on the named entities. This is by far the slowest part of my code, partially becuase of the operation involved, and partially becuase the NLTK Stanford-NER function is a wrapper running the original Java version in the background. 

##### Evaluating these features

In order for these features to be meaningful and helpful, they should separate irrelevant and relevant pages. In order to double check this, I plotted histograms, grouped by relevant/irrelevance. Better measures pull the green more to the right, and the blue more to the left. For no good reason, the order in which I created the features (which is the order in which I present them here) is also their order of importance. Maybe it just speaks to the most obvious thing being the best thing. 

### cosine similarity
Histogram of title cosine similarity:
<img src = "http://noahburbank.com/files/title%20similarity.png">
Histogram of body cosine similarity:
<img src = "http://noahburbank.com/files/body%20sim.png">
Histogram of description similarity:
<img src = "http://noahburbank.com/files/desc%20sim.png">

### one gram
Histogram of body one gram similarity:
<img src = "http://noahburbank.com/files/body%20one%20gram.png">
Histogram of body two gram similarity:
<img src = "http://noahburbank.com/files/body%20two%20gram.png">
Histogram of body three gram similarity:
<img src = "http://noahburbank.com/files/body%20three%20gram.png">

### one gram
Histogram of description one gram similarity:
<img src = "http://noahburbank.com/files/desc%20one%20gram.png">
Histogram of description two gram similarity:
<img src = "http://noahburbank.com/files/desc%20two%20gram.png">
Histogram of description three gram similarity:
<img src = "http://noahburbank.com/files/desc%20three%20gram.png">

### longest common string
Histogram of body longest common string:
<img src = "http://noahburbank.com/files/body%20lcf.png">
Histogram of title longest common string:
<img src = "http://noahburbank.com/files/title%20lcf.png">
Histogram of description longest common string:
<img src = "http://noahburbank.com/files/desc%20lcf.png">

I'll stop here, since the features are less and less valuable, but if you care to see more please, look at: [4 signal extract](4 signal extraction.ipynb)

## 2 Training the model 

I divided my remaining into 60% training and 40% test set. Becuase of the data set size, and the number of features that I created, I wanted to avoid overfiting the data, so I wanted to give a pretty big holdout set. The second reason is that I didn't just want global performance, I wanted performance within the 26 webdomains represented by the publisher urls, so it was critical that both the training and testing sets for each domain were sufficiently large. 

For fun, I ran a variety of models, but at the end of the day there are only two models that I care about: the random forests model and the logistic regression model. The random forests model has the best performance, and the logistic model is really fast (about two orders of magnitude faster to train than random forests). 

<img src = "http://noahburbank.com/files/all%20auc.png">

I chose pretty basic setting for random forests. In order to capture the potential diversity of behavior across publisher web domains, I trained 100 estimators, but to keep from overfitting I allowed a maximum of 5 features to be considered. Its performance is pretty good. 

## 3 Testing the model 

I want my model to perform well for any given pub/ad url pair and for any domain. In order to assess the first, we can just look to the standard assessments: AUC of .973, cross validation accuracy of .817, holdout F1 score of .872. 

In order to assess the latter, we need to look at performance across all 26 publisher web domains. If we look at a histogram of the F1 scores, they are generally excellent, but I was concerned about the few domains on which the F1 score was 0.4. In order to get a better insight into this, I ran a simulate by iterating through two hundred seed values, and then plotting the histogram of within-domain F1 score (hence the values on the Y axis). In order to figure out what was going wrong, I plotted these F1 values against the percentage of irrelevant pub/ad pairs for each domain. What I conclude is that, although sometimes the F1 score is low, this it is below 0.7 less than .3% of the time, and it is above 0.95 83% of the time.

<img src= "http://noahburbank.com/files/f1%20vs.%20irr%20percent.png">

At first I was a bit worried about how good my model's performance is. I thought about it a bit, and even re-ran everything with the first experiment, which also had really good performance. Then I realzied what it was: by defining relevance as publisher and advertiser pages containing the same named entities, I was working with a machine learnable definition of relevance. With this in mind, it should be no surprised that performance was so high, it should be a surprised that NER was so unimportant, but given how much semantic value is captured by tf-idf and cosine similarity just in the title, there was little left for NER to do. 

For more on my model and it's validation, please see [5 model](5 model v2.1.ipynb)